# Market Regimes and Financial Interpretation

## Introduction

Financial markets do not move in a smooth or uniform manner over time. Instead, they tend to operate in distinct phases where market behavior remains relatively stable before shifting to a new phase. These phases are commonly referred to as market regimes. In this notebook, we analyze NIFTY 50 index data to examine whether such regime-like behavior is visible through simple financial indicators such as returns, volatility, and drawdowns


## Understanding Market Regimes

A market regime is a sustained phase during which market behavior remains relatively stable in its statistical and structural characteristics. Rather than moving in a smooth or uniform manner, financial markets shift between distinct states defined by varying levels of risk, momentum, volatility, and investor sentiment.

### Return Distribution

Different regimes produce distinct return characteristics:
Bull regimes typically show positive average returns with mild right skewness.
Bear regimes exhibit negative average returns and greater downside asymmetry.
Range-bound regimes fluctuate around zero with relatively thin tails.
Crisis regimes display fat tails, meaning extreme moves occur more frequently than expected.

## Data Description

The analysis is based on daily NIFTY 50 index data. The dataset includes:
- Open, high, low, and close prices
- Trading volume (shares traded)
- Market turnover

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")
df = pd.read_csv("cleaned_nse_data.csv")
df["trade_date"] = pd.to_datetime(df["trade_date"])
df = df.sort_values("trade_date").reset_index(drop=True)
df.head()

### Explanation of Columns

- **trade_date**: The trading date for the observation.
- **open**: Index level at market open.
- **high**: Highest index level reached during the trading day.
- **low**: Lowest index level reached during the trading day.
- **close**: Index level at market close.
- **shares_traded**: Total number of shares traded across all constituent stocks.
- **turnover_cr**: Total traded value in crore INR, reflecting market liquidity and participation.

Prices tell us where the market went, while volume and turnover tell us how intensely people were trading. Looking at both together helps us understand not just price changes, but also investor behavior behind those moves.

## Price Dynamics and Market Movement

The open, high, low, and close prices describe how the market moves within a trading day.
Large differences between high and low prices indicate intraday uncertainty, while consistent
closing trends over time reflect broader market direction.

By analyzing closing prices over time, we can observe long-term growth phases, stagnation
periods, and sharp corrections, all of which are characteristic of different market regimes.


In [ ]:
plt.figure(figsize=(12,4))
plt.plot(df["trade_date"], df["close"])
plt.title("NIFTY 50 Closing Index Level")
plt.xlabel("Date")
plt.ylabel("Index Level")
plt.show()


The closing index represents the final market value at the end of each day.
The graph shows steady growth most of the time, with sudden drops during specific periods, suggesting that the market goes through different phases.

## Return Behavior

While prices show long-term trends, returns capture day-to-day changes in market value.
Analyzing returns helps reveal periods of increased uncertainty that are often associated with changes in market conditions.


In [ ]:
df["log_return"] = np.log(df["close"] / df["close"].shift(1))
df.dropna(inplace=True)

plt.figure(figsize=(12,4))
plt.plot(df["trade_date"], df["log_return"], alpha=0.6)
plt.title("Daily Log Returns of NIFTY 50")
plt.xlabel("Date")
plt.ylabel("Log Return")
plt.show()


Daily returns fluctuate often, but major gains and losses usually happen at similar times. This suggests the market has calm times and more uncertain times rather than acting completely at random.

## Volatility Clustering

Volatility shows how risky the market is. When the market is calm, volatility stays about the same over time. But in real situations, volatility tends to group together in long periods of low and high risk, which is an important feature of how markets behave in different stages.

In [ ]:
df["rolling_vol_21d"] = df["log_return"].rolling(window=21).std()

plt.figure(figsize=(12,4))
plt.plot(df["trade_date"], df["rolling_vol_21d"], color="red")
plt.title("21-Day Rolling Volatility of NIFTY 50")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.show()


## Drawdown Analysis


Drawdowns measure how much the market has fallen from its previous highest level.
They capture the depth and duration of losses experienced by investors and are
useful for identifying periods of severe market stress.


In [ ]:

df["cum_return"] = (1 + df["log_return"]).cumprod()
df["cum_max"] = df["cum_return"].cummax()
df["drawdown"] = (df["cum_return"] - df["cum_max"]) / df["cum_max"]

plt.figure(figsize=(12,4))
plt.plot(df["trade_date"], df["drawdown"], color="black")
plt.title("NIFTY 50 Drawdown")
plt.xlabel("Date")
plt.ylabel("Drawdown")
plt.show()


## Event-Based Analysis: COVID-19 Market Crash


Major market events often trigger abrupt changes in market behavior.
The COVID-19 pandemic provides a clear real-world example of a sudden
transition from a relatively stable market phase to a high-stress regime.


In [ ]:
#isolating 2020-2021
covid_df = df[
    (df["trade_date"] >= "2020-01-01") & 
    (df["trade_date"] <= "2021-12-31")
]

plt.figure(figsize=(12,4))
plt.plot(covid_df["trade_date"], covid_df["rolling_vol_21d"], color="red")
plt.title("Rolling Volatility During COVID-19 Period")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.show()


During the COVID period, volatility rises sharply compared to surrounding years.
This spike reflects sudden uncertainty and panic-driven trading as markets reacted
to unprecedented global disruptions. The rapid shift from low to extremely high
volatility illustrates a clear regime transition rather than a gradual change.


In [ ]:
#drawdown
plt.figure(figsize=(12,4))
plt.plot(covid_df["trade_date"], covid_df["drawdown"], color="black")
plt.title("Drawdown During COVID-19 Period")
plt.xlabel("Date")
plt.ylabel("Drawdown")
plt.show()


The drawdown during the COVID period shows a sharp and deep decline from previous
market highs. This confirms that the volatility spike corresponds to a severe
crisis regime with significant losses before recovery begins.


## Other Major Market Stress Periods


Apart from COVID-19, the market exhibits several other periods of elevated volatility
and drawdowns across the full dataset. These episodes correspond to global or domestic
financial stress events, where market behavior temporarily shifts into high-risk regimes.

Although the causes differ, the market response is similar across events, reinforcing
the idea that markets cycle through recurring regimes rather than reacting in a purely
random manner.


## Market Regime Classification


Market regimes can be broadly classified by combining information from price trends,
volatility levels, and drawdowns. Rather than assigning explicit labels, we interpret
regimes based on observable market behavior.

Bull regimes are characterized by rising prices, relatively low volatility, and
small drawdowns. Bear or crisis regimes are marked by falling prices, sharp increases
in volatility, and deep drawdowns. Transitional periods lie between these extremes
and reflect changing market conditions.


### Regime Characteristics Summary

| Regime Type | Price Trend | Volatility | Drawdown |
|------------|------------|------------|----------|
| Bull | Sustained upward movement | Low | Small |
| Bear | Declining or unstable prices | High | Moderate |
| Crisis | Sharp price drops | Very high | Large |


The previous sections show that these regime characteristics appear repeatedly in the
data. Long growth phases with low volatility are interrupted by periods of heightened
risk, where volatility spikes and drawdowns deepen. Major events such as the COVID-19
crash represent extreme examples of crisis regimes, while calmer periods correspond
to bull regimes.

This behavior indicates that market regimes are persistent and distinct, rather than
short-lived or random.


## Key Takeaways and Implications


- The NIFTY 50 index exhibits clear regime-like behavior across time.
- Price trends, return clustering, volatility spikes, and drawdowns jointly reveal
  persistent market phases.
- Periods of market stress are marked by abrupt transitions rather than gradual changes.
- Real-world events such as the COVID-19 crash validate the existence of distinct
  crisis regimes.

These findings motivate the use of regime-switching models such as Hidden Markov Models
and deep temporal clustering approaches, which are designed to capture persistent market
states and transitions that are not explained by stationary models.
